In [1]:
import sys
import json
import csv
import os
from os import listdir

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [2]:
sys.path.append('..')

from utils import get_project_root

In [11]:
file_suffixes = []
file_data = []
yamnet_embeddings_dir = os.path.join(
    get_project_root(), "audioset_yamnet_embeddings/val/")

for file in os.listdir(yamnet_embeddings_dir):
    if file.endswith("_embeddings.npy"):
        file_suffixes.append(file[1:12])

        x = np.load(yamnet_embeddings_dir + file)
        x = np.pad(x, ((0, 20-x.shape[0]), (0, 0)), mode='edge')

        file_data.append(x)

suffix_to_label_id = {}
df = pd.read_csv(os.path.join(get_project_root(), 'metadata/unbalanced_train_segments.csv'),
                 sep=',', quotechar='"', skipinitialspace=True, skiprows=2)

labels = []
for i in range(len(file_suffixes)):
    # print(file_suffixes[i])
    labels.append(df.loc[df['# YTID'] == file_suffixes[i],
                  'positive_labels'].values.item())

print(len(file_suffixes))
print(len(labels))
print(labels)


12676
12676
['/m/07qlwh6,/m/09x0r,/t/dd00125', '/m/01g90h,/m/09x0r', '/m/03xq_f,/m/04rlf', '/m/015p6,/m/020bb7,/m/09d5_', '/m/028v0c,/m/0cmf2,/m/0k5j,/t/dd00129', '/m/02cz_7,/m/04rlf,/m/09x0r', '/m/04rlf,/m/07k1x', '/m/02g901', '/m/0145m,/m/04rlf,/m/09x0r', '/m/03qtwd', '/m/012f08,/m/04_sv,/m/07yv9', '/m/02bk07', '/m/07pp_mv', '/m/012f08,/m/04qvtq,/m/07yv9,/m/0k4j,/t/dd00134', '/m/0l156k', '/m/0326g,/m/03qtwd,/m/04rlf,/m/09x0r', '/m/01v_m0,/m/0hdsk', '/m/07pdjhy', '/m/07bjf,/m/09x0r', '/m/0k4j,/t/dd00134', '/m/01m4t', '/m/01kcd,/m/06ncr', '/m/04rlf,/m/04szw,/m/07xzm,/m/0fx80y', '/m/027m70_', '/m/04rlf,/m/0fd3y', '/m/028v0c,/m/07pc8lb', '/m/01xq0k1,/m/0ch8v', '/m/07ppn3j,/m/09x0r,/m/0ytgt,/t/dd00125', '/m/07s12q4,/m/09x0r', '/m/09x0r,/m/0c2wf,/t/dd00127', '/m/02y_763', '/m/081rb', '/m/02cjck,/m/04rlf', '/m/01280g,/m/09x0r,/m/0jbk', '/m/07swgks,/m/09x0r', '/m/018vs,/m/0342h,/m/04rlf,/m/04szw,/m/0fx80y', '/m/03wvsk', '/m/04rlf,/m/07kc_', '/m/01xq0k1,/m/07rpkh9,/m/0ch8v', '/m/01jnbd', '/m/

In [12]:
def get_all_children(category,aso):
    
    childs = aso[category]["child_ids"]
    childs_names = []
    for child in childs:
        child_name = {}
        child_name["name"] = aso[child]["name"]
        if "child_ids" in aso[child]: child_name["children"] = get_all_children(child,aso)
        childs_names.append(child_name)
    if childs_names : return childs_names


def preprocess_ontology(file_dir):
    
    f = open(file_dir)
    ont_data = json.load(f)
    f.close()

    ont = {}
    for category in ont_data:
        tmp = {}
        tmp["name"] = category["name"]
        tmp["restrictions"] = category["restrictions"]
        tmp["child_ids"] = category["child_ids"]
        tmp["parents_ids"] = []
        ont[category["id"]] = tmp

    for category in ont: # find parents
        for c in ont[category]["child_ids"]:
            ont[c]["parents_ids"].append(category)

    higher_categories=[] # higher_categories are the ones without parents
    for category in ont: 
        if ont[category]["parents_ids"] == []:
            higher_categories.append(category)
            
    return ont, higher_categories
        
def get_all_parents(id_, ont, parents_names):
    
    parents = ont[id_]["parents_ids"]
    
    if parents:
        for parent in parents:
            parent_name = ont[parents[0]]["name"]        
            get_all_parents(parents[0], ont, parents_names)
    
    parents_names.append(ont[id_]["name"])

In [13]:
index_to_id = {}
index_to_classname = {}
with open(os.path.join(get_project_root(), 'metadata/class_labels_indices.csv'), mode='r') as file:
    reader = csv.reader(file)    
    for rows in reader:
        index_to_id[rows[0]] = rows[1]
        index_to_classname[rows[0]] = rows[2]

# Data/class arrays
train_class_1 = []
train_class_2 = []

# Unique level 1 and 2 classes
class_1 = []
class_2 = []
ont, highest_category = preprocess_ontology(os.path.join(get_project_root(), 'metadata/ontology.json'))
for id_ in highest_category:
    class_2.append(ont[id_]['name'])
    for id1_ in ont[id_]['child_ids']:
        class_1.append(ont[id1_]['name'])

class_1 = np.unique(class_1)
class_1 = np.concatenate((class_1[:29], class_1[30::]))
class_2 = np.unique(class_2) 

class_to_index_1 = {class_1[i]:i for i in range(len(class_1))}
class_to_index_2 = {class_2[i]:i for i in range(len(class_2))}

print(class_to_index_1)


        

{'Acoustic environment': 0, 'Alarm': 1, 'Bell': 2, 'Deformable shell': 3, 'Digestive': 4, 'Domestic animals, pets': 5, 'Domestic sounds, home sounds': 6, 'Engine': 7, 'Explosion': 8, 'Fire': 9, 'Generic impact sounds': 10, 'Glass': 11, 'Hands': 12, 'Heart sounds, heartbeat': 13, 'Human group actions': 14, 'Human locomotion': 15, 'Human voice': 16, 'Liquid': 17, 'Livestock, farm animals, working animals': 18, 'Mechanisms': 19, 'Miscellaneous sources': 20, 'Music genre': 21, 'Music mood': 22, 'Music role': 23, 'Musical concepts': 24, 'Musical instrument': 25, 'Noise': 26, 'Onomatopoeia': 27, 'Other sourceless': 28, 'Respiratory sounds': 29, 'Silence': 30, 'Sound reproduction': 31, 'Specific impact sounds': 32, 'Surface contact': 33, 'Thunderstorm': 34, 'Tools': 35, 'Vehicle': 36, 'Water': 37, 'Whistling': 38, 'Wild animals': 39, 'Wind': 40, 'Wood': 41}


In [14]:
val_class_1 = []
val_class_2 = []

for lbl in labels:
    # print(lbl)
    ids = lbl.split(',')
    parent_names = []
    for id_ in ids:
        get_all_parents(id_, ont, parent_names)
    
    parent_names = np.unique(parent_names)
        
    class_1_label = list(set(class_1) & set(parent_names))
    class_2_label = list(set(class_2) & set(parent_names))
    # print(class_1_label)

    target_1_label = []
    target_2_label = []
                                            
    for class_ in class_1_label:
        target_1_label.append(class_to_index_1[class_])
    for class_ in class_2_label:
        target_2_label.append(class_to_index_2[class_])

    val_class_1.append(np.asarray(target_1_label))
    val_class_2.append(np.asarray(target_2_label))

In [15]:
print(len(val_class_1))
print(len(val_class_2))
print(val_class_1)
# print(np.concatenate(file_data).shape)
# print(len(file_data))

12676
12676
[array([17, 16,  0]), array([16,  4]), array([25]), array([39]), array([30,  0, 36]), array([16, 21]), array([35]), array([6]), array([16, 21]), array([14]), array([36]), array([16]), array([1]), array([36]), array([1]), array([14, 16, 21]), array([28]), array([33]), array([16, 19]), array([36]), array([19]), array([25]), array([25]), array([ 2, 25]), array([21]), array([10, 30]), array([18]), array([29, 16,  0]), array([16, 27]), array([ 6, 16,  0]), array([6]), array([6]), array([23]), array([39, 16]), array([37, 16]), array([25]), array([6]), array([25]), array([18]), array([0]), array([26]), array([19]), array([14]), array([6, 9]), array([ 2, 25]), array([21]), array([36, 16]), array([16, 25, 21]), array([23]), array([16]), array([25, 21]), array([21]), array([6]), array([17]), array([29, 16]), array([5]), array([36]), array([11]), array([25]), array([21]), array([14]), array([19]), array([29]), array([ 6, 16,  0]), array([16, 32]), array([16,  4]), array([12]), array([

In [6]:
yamnet_embeddings_dir = os.path.join(get_project_root(), "audioset_yamnet_embeddings/val/")

filenames = os.listdir(yamnet_embeddings_dir)
file_data = []
for file in filenames:
    if file.endswith("_embeddings.npy"):
        x = np.load(yamnet_embeddings_dir + file)
        x = np.pad(x, ((0, 20-x.shape[0]), (0, 0)), mode='edge')
    
        file_data.append(x)
        
        

In [ ]:
# Save all data files
save_dir = os.path.join(get_project_root(), 'data')
np.save(os.path.join(save_dir, 'audioset_yamnet_train_data.npy'), np.asanyarray(file_data, dtype=float))
np.save(os.path.join(save_dir, 'audioset_yamnet_train_labels_1.npy'), np.asanyarray(train_class_1, dtype=object))
np.save(os.path.join(save_dir, 'audioset_yamnet_train_labels_2.npy'), np.asanyarray(train_class_2, dtype=object))

In [ ]:
plt.figure()
plt.hist(file_data[0])
plt.show()

print(x.shape)

In [16]:
data = np.load(os.path.join(get_project_root(), 'data/audioset_yamnet_val_data.npy'), allow_pickle=True)
val_class_1 = np.load(os.path.join(get_project_root(), 'data/audioset_yamnet_val_labels_1.npy'), allow_pickle=True)
val_class_2 = np.load(os.path.join(get_project_root(), 'data/audioset_yamnet_val_labels_2.npy'), allow_pickle=True)
data = file_data
val_class_1 = train_class_1
val_class_2 = train_class_2

num_val = 0.20 * 21508
num_class_1 = len(class_1) 
num_val_per_class = int(num_val / num_class_1)

val_data = []
np.random.seed(0)

num_clips = len(data)
logits_1 = np.zeros((num_clips, num_class_1))

for i in range(num_clips):
    logits_1[i][val_class_1[i].astype(int)] = 1
    
val_class1_subset = []
val_class2_subset = []

for idx in range(num_class_1):
    
    idx_match = np.where( logits_1[:, idx] == 1)[0]
    if idx_match.size != 0:
        val_idx = np.random.choice(idx_match, num_val_per_class)

    for i in range(num_val_per_class):
        val_data.append(data[val_idx[i]])
        val_class1_subset.append(val_class_1[val_idx[i]].astype(int))
        val_class2_subset.append(val_class_2[val_idx[i]].astype(int))
    

val_data_padded = []
for x in val_data:
    x = np.pad(x, ((0, 20-x.shape[0]), (0, 0)), mode='edge')
    val_data_padded.append(x)

    
# print(len(labels1))
print(len(file_data))
print(len(np.concatenate(file_data)))


12676
253520


In [17]:
# file_data = np.load(os.path.join(get_project_root(), 'data/audioset_yamnet_val_data.npy'), allow_pickle=True)
# val_class_1_subset = np.load(os.path.join(get_project_root(), 'data/audioset_yamnet_val_labels_1.npy'), allow_pickle=True)
# val_class_2_subset = np.load(os.path.join(get_project_root(), 'data/audioset_yamnet_val_labels_2.npy'), allow_pickle=True)


# Save all data files
save_dir = os.path.join(get_project_root(), 'data')
np.save(os.path.join(save_dir, 'audioset_yamnet_val_data.npy'), np.asanyarray(val_data_padded, dtype=float))
np.save(os.path.join(save_dir, 'audioset_yamnet_val_labels_1.npy'), np.asanyarray(val_class1_subset, dtype=object))
np.save(os.path.join(save_dir, 'audioset_yamnet_val_labels_2.npy'), np.asanyarray(val_class2_subset, dtype=object))


In [20]:
print(len(val_data_padded))
print(len(val_class1_subset))

4284
4284


In [ ]:
train_labels1 = np.load(os.path.join(get_project_root(), 'data/audioset_yamnet_train_labels_1.npy'), allow_pickle=True)
train_labels2 = np.load(os.path.join(get_project_root(), './data/audioset_yamnet_train_labels_2.npy'), allow_pickle=True)
train_data = np.load(os.path.join(get_project_root(), './data/audioset_yamnet_train_data.npy'), allow_pickle=True)

# train_labels1 = []
# for lbl in labels1:
#     train_labels1.append(lbl.astype('int'))
# train_labels2 = []
# for lbl in labels2:
#     train_labels2.append(lbl.astype('int'))

save_dir = os.path.join(get_project_root(), 'data')

np.save(os.path.join(save_dir, 'audioset_yamnet_train_data.npy'),
        np.asanyarray(train_data, dtype=float))
np.save(os.path.join(save_dir, 'audioset_yamnet_train_labels_1.npy'),
        np.asanyarray(train_labels1, dtype=int))
np.save(os.path.join(save_dir, 'audioset_yamnet_train_labels_2.npy'),
        np.asanyarray(train_labels2, dtype=int))
